In [16]:
import logging
from copy import copy
from enum import Enum
import matplotlib.pyplot as plt
import cv2
import scipy

import numpy as np
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, AngleAxis,
                         Concatenate, DiagramBuilder, InputPortIndex,
                         LeafSystem, MeshcatVisualizer, Parser,
                         PiecewisePolynomial, PiecewisePose, PointCloud,
                         PortSwitch, RandomGenerator, RigidTransform,
                         RollPitchYaw, Simulator, StartMeshcat,
                         UniformlyRandomRotationMatrix, FindResourceOrThrow, Sphere,
                         RotationMatrix, SpatialVelocity, InverseDynamicsController,
                         MultibodyPlant, PassThrough, Demultiplexer, Adder, StateInterpolatorWithDiscreteDerivative,
                         MathematicalProgram, Solve, PointCloud, ConstantVectorSource, Integrator, DiscreteDerivative,
                         DifferentialInverseKinematicsParameters, DifferentialInverseKinematicsIntegrator,
                         InverseKinematics)

from manipulation import FindResource, running_as_notebook
from manipulation.clutter import GenerateAntipodalGraspCandidate
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.pick import (MakeGripperCommandTrajectory, MakeGripperFrames,
                               MakeGripperPoseTrajectory)
from manipulation.scenarios import (AddIiwaDifferentialIK, AddPackagePaths, AddRgbdSensors,
                                    MakeManipulationStation, ycb, AddShape, AddIiwa)
from manipulation.utils import AddPackagePaths, FindResource

class NoDiffIKWarnings(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith('Differential IK')

logging.getLogger("drake").addFilter(NoDiffIKWarnings())

TIME_STEP = 0.05
CAM_NOISE = 0.003

In [2]:
def AddBin(plant,
           iiwa_model_instance):
    parser = Parser(plant)
    gripper = parser.AddModelFromFile(
                "../manipulation/manipulation/models/bin_small.sdf")

    X_7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, 0), [0.0, 0.02625, 0.12])
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa_model_instance),
                     plant.GetFrameByName("bin_base", gripper), X_7G)
    return gripper

In [3]:
def AddFastIiwaDifferentialIK(builder, plant, frame=None):
    params = DifferentialInverseKinematicsParameters(plant.num_positions(),
                                                     plant.num_velocities())
    time_step = plant.time_step()
    q0 = plant.GetPositions(plant.CreateDefaultContext())
    params.set_nominal_joint_position(q0)
    params.set_end_effector_angular_speed_limit(5)
    params.set_end_effector_translational_velocity_limits([-5, -5, -5],
                                                          [5, 5, 5])
    if plant.num_positions() == 3:  # planar iiwa
        iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits))
        # These constants are in body frame
        assert (
            frame.name() == "iiwa_link_7"
        ), "Still need to generalize the remaining planar diff IK params for different frames"  # noqa
        params.set_end_effector_velocity_flag(
            [True, False, False, True, False, True])
    else:
        iiwa14_velocity_limits = np.ones(7) * 5.0 #np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits))
        params.set_joint_centering_gain(10 * np.eye(7))
    if frame is None:
        frame = plant.GetFrameByName("body")
    differential_ik = builder.AddSystem(
        DifferentialInverseKinematicsIntegrator(
            plant,
            frame,
            time_step,
            params,
            log_only_when_result_state_changes=True))
    return differential_ik

In [4]:
def compute_sphere(a, b, c, d):
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(3, 'x')
    prog.AddLinearEqualityConstraint((x - (a + b) / 2).dot(b - a), 0)
    prog.AddLinearEqualityConstraint((x - (c + b) / 2).dot(b - c), 0)
    prog.AddLinearEqualityConstraint((x - (c + d) / 2).dot(d - c), 0)
    res = Solve(prog)
    if res.is_success:
        center = res.GetSolution()
        radius = np.linalg.norm(center - a)
        return np.array([center[0], center[1], center[2], radius])
    
    return None

In [6]:
def icp_find_sphere(point_cloud, num_iters = 500, tol = 0.001):
    best_sol, num_inliers = None, 0
    pts = point_cloud.xyzs()
    pts = pts[:, pts[0, :] != np.inf]
    pts += np.random.randn(*pts.shape) * CAM_NOISE
    if pts.shape[1] < 30:
        return None
    
    for _ in range(num_iters):
        inds = np.random.choice(pts.shape[1], size=4)
        new_sol = compute_sphere(pts[:, inds[0]], pts[:, inds[1]], pts[:, inds[2]], pts[:, inds[3]])
        if new_sol is None:
            continue
            
        pt_dists = np.linalg.norm(pts - new_sol[:3, None], axis=0) - new_sol[3]
        new_inliers = np.sum(np.abs(pt_dists) < tol)
        if new_inliers > num_inliers:
            best_sol, num_inliers = new_sol, new_inliers
    return best_sol

def prog_find_sphere_iter(point_cloud, num_iters = 500, tol = 0.001):
    pts = point_cloud.xyzs()
    pts = pts[:, pts[0, :] != np.inf]
    pts += np.random.randn(*pts.shape) * CAM_NOISE
    if pts.shape[1] < 30:
        return None
    
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(3, 'x')
    r = 0.02
    
    for i in range(pts.shape[1]):
        for j in range(pts.shape[1]):
            p1 = pts[:, i]
            p2 = pts[:, j]
            dist = (p1 - x).dot(p1 - x) - (p2 - x).dot(p2 - x)
            prog.AddCost(dist * dist / pts.shape[1])
        
    res = Solve(prog)
    if res.is_success():
        #print("FOUND")
        #print(res.GetSolution())
        return np.concatenate((res.GetSolution(), np.array([r])))
    else:
        return None
    
SPHERE_FINDER = prog_find_sphere_iter
        

In [7]:
class ICPSphereFinder(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        
        model_point_cloud = AbstractValue.Make(PointCloud(0))

        self.DeclareAbstractInputPort("point_cloud", model_point_cloud)
        self.DeclareVectorOutputPort("sphere_pos", 4,
                                     self.CalcOutput)
        
        self.count_ind = self.DeclareDiscreteState(1)

    def CalcOutput(self, context, output):
        if context.get_discrete_state().get_value() < 2:
            cloud = self.get_input_port().Eval(context)
            sphere = SPHERE_FINDER(cloud, 50, 0.005)
            if sphere is not None:
                output.SetFromVector(sphere)
                context.get_mutable_discrete_state().set_value(context.get_discrete_state().get_value() + 1)
            else:
                output.SetFromVector(np.ones(4) * np.nan)
        else:
            output.SetFromVector(np.ones(4) * np.nan)
            
class TrajectoryEstimator(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self._g = np.array([0, 0, 9.81])
        
        self.DeclareVectorInputPort("sphere_position", 4)
        self.DeclareVectorInputPort("time", 1)
        self.DeclareVectorOutputPort("trajectory_estimate", 6, self.CalcOutput)
        
        self.estimate_ind = self.DeclareDiscreteState(6)
        self.prev_pos_ind = self.DeclareDiscreteState(np.ones(4) * np.nan) # xyz then t
        
        self.DeclarePeriodicDiscreteUpdateEvent(TIME_STEP, 0.0, self.Update)
    
    def Update(self, context, state):
        sphere_pos = self.get_input_port(0).Eval(context)[:3]
        time = self.get_input_port(1).Eval(context)[0]
        
        
        cur_est = state.get_mutable_value(self.estimate_ind)
        prev = state.get_mutable_value(self.prev_pos_ind)
        prev_pos = prev[:3]
        prev_time = prev[3]
        
        
        if not np.any(np.isnan(sphere_pos)) and not np.any(np.isnan(prev_pos)):
            
            # correct for lag
            time_diff = time - prev_time
            aimed_pos = sphere_pos + self._g / 2 * time_diff * time_diff
            old_vel = (aimed_pos - prev_pos) / time_diff
            sphere_vel = old_vel - self._g * time_diff
            
            start_pos = sphere_pos + time * (-sphere_vel) - self._g / 2 * time * time
            start_vel = sphere_vel + self._g * time
            
            print("Predicted trajectory:", start_pos, start_vel)

            state_est = np.zeros(6)
            state_est[:3] = start_vel
            state_est[3:] = start_pos
            
            if np.linalg.norm(cur_est) < 0.01:
                cur_est = state_est
            else:
                cur_est = cur_est * 0.7 + state_est * 0.3
                
        state.set_value(self.estimate_ind, cur_est)
        if not np.any(np.isnan(sphere_pos)):
            state.set_value(self.prev_pos_ind, np.array([sphere_pos[0], sphere_pos[1], sphere_pos[2], time]))
        #print(sphere_pos, sphere_vel, cur_est, time)
            
    
    def CalcOutput(self, context, output):
        output.SetFromVector(context.get_discrete_state(self.estimate_ind).get_value())
        
class CatchPlanner(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._g = np.array([0, 0, 9.81])
        self._plant = plant
        self._bin_body_ind = plant.GetBodyByName("bin_base").index()
        self.DeclareVectorInputPort("estimated_trajectory", 6)
        self.DeclareAbstractInputPort("body_poses", AbstractValue.Make([RigidTransform()]))
        self.int_id = self.DeclareDiscreteState(7)
        self.DeclareVectorOutputPort("interception_data", 6, self.GetOutput)
        self.DeclareVectorOutputPort("interception_time", 1, self.GetTime)
        
        self.DeclarePeriodicUnrestrictedUpdateEvent(TIME_STEP, 0.0, self.Update)
        
        
    def Update(self, context, state):
        traj = self.get_input_port(0).Eval(context)
        traj_vel = traj[:3]
        traj_pos = traj[3:]
        
        if np.linalg.norm(traj) < 0.01:
            state.get_mutable_discrete_state().set_value(np.zeros(7))
        elif not np.all(state.get_discrete_state().get_value() == 0):
            return
        else:
            body_poses = self.get_input_port(1).Eval(context)
            bin_pos = body_poses[self._bin_body_ind].translation()
            prog = MathematicalProgram()
            t = prog.NewContinuousVariables(1, 'x')[0]
            x = traj_pos + t * traj_vel - self._g / 2 * t * t
            prog.AddCost((x - bin_pos).dot(x - bin_pos))
            prog.AddConstraint(t >= 0)
            #prog.AddConstraint(x[:2].dot(x[:2]) + 0.01 * x[2] * x[2] >= 0.1)
            
            res = Solve(prog)
            t_int = res.GetSolution()
            x_int = traj_pos + t_int * traj_vel - self._g / 2 * t_int * t_int
            v_int = traj_vel - self._g * t_int
            print("Intercept at time", t_int[0], x_int)
            state.get_mutable_discrete_state().set_value(np.concatenate((x_int, -v_int, t_int + 0.05)))
            
    def GetOutput(self, context, output):
        output.set_value(context.get_discrete_state().get_value()[:6])
        
    def GetTime(self, context, output):
        output.set_value(context.get_discrete_state().get_value()[6:])
        
class CatchIKcontroller(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._g = np.array([0, 0, 9.81])
        self.DeclareVectorInputPort("interception_data", 6)
        self.DeclareVectorInputPort("iiwa_position", 7)
        self.DeclareVectorOutputPort("catch_position", 7, self.GetOutput)
        
        self.cache_id = self.DeclareDiscreteState(7)
        
    def GetOutput(self, context, output):
        cached_output = context.get_discrete_state(self.cache_id).get_value()
        int_data = self.get_input_port(0).Eval(context)
        int_pos, int_ori = int_data[:3], int_data[3:]
        
        iiwa_position = self.get_input_port(1).Eval(context)
        
        
        
        if np.all(cached_output == 0):
            if np.all(int_data == 0):
                output.set_value(iiwa_position)
            else:
                ik = InverseKinematics(self._plant)
                bin_frame = self._plant.GetFrameByName('bin_base')
                
                
                target_ori = ([0, 0, 1] + (int_ori / np.linalg.norm(int_ori))) / 2
                
                ik.AddPositionConstraint(
                    bin_frame, [0, 0, 0.015], self._plant.world_frame(),
                    int_pos, int_pos)
                
                ik.AddAngleBetweenVectorsConstraint(
                    bin_frame, [0, 0, 1], self._plant.world_frame(), int_ori, 0, 0)
                
#                 ik.AddAngleBetweenVectorsCost(bin_frame, [0, 0, 1], self._plant.world_frame(), [0, 0, 1], 1)
                
                prog = ik.get_mutable_prog()
                q = ik.q()
                prog.AddQuadraticErrorCost(np.identity(7), iiwa_position, q[:7])
                prog.SetInitialGuess(q[:7], iiwa_position)
                result = Solve(ik.prog())
                
                if result.is_success():
                    sol = result.GetSolution()
                    context.get_mutable_discrete_state(self.cache_id).set_value(sol[:7])
                    output.set_value(sol[:7])
                else:
                    print("IK Failure!")
                    context.get_mutable_discrete_state(self.cache_id).set_value(iiwa_position)
                    output.set_value(iiwa_position)
                    
        else:
            output.set_value(cached_output)
                

In [8]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [18]:
meshcat.Delete()
meshcat.DeleteAddedControls()
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
parser = Parser(plant, scene_graph)
AddPackagePaths(parser)

# Note that we parse into both the plant and the scene_graph here.
iiwa_model, cam_model, bin_model = parser.AddModels(
        "../manipulation/manipulation/models/catching_env.dmd.yaml")

projectile = AddShape(plant, Sphere(0.02), "projectile")
proj_body = plant.GetBodyByName("projectile")
plant.SetDefaultFreeBodyPose(proj_body, RigidTransform(RotationMatrix.Identity(), [-2, 0, 0]))

plant.Finalize()
AddRgbdSensors(builder, plant, scene_graph)

# Adds the MeshcatVisualizer and wires it to the SceneGraph.
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

num_iiwa_positions = plant.num_positions(iiwa_model)
model_instance_name = "iiwa"
model_instance = iiwa_model

# I need a PassThrough system so that I can export the input port.
iiwa_position = builder.AddSystem(PassThrough(num_iiwa_positions))
# builder.ExportInput(iiwa_position.get_input_port(),
#                     model_instance_name + "_position")
builder.ExportOutput(iiwa_position.get_output_port(),
                     model_instance_name + "_position_commanded")

# Export the iiwa "state" outputs.
demux = builder.AddSystem(
    Demultiplexer(2 * num_iiwa_positions, num_iiwa_positions))
builder.Connect(plant.get_state_output_port(model_instance),
                demux.get_input_port())
builder.ExportOutput(demux.get_output_port(0),
                     model_instance_name + "_position_measured")
builder.ExportOutput(demux.get_output_port(1),
                     model_instance_name + "_velocity_estimated")
builder.ExportOutput(plant.get_state_output_port(model_instance),
                     model_instance_name + "_state_estimated")

# Make the plant for the iiwa controller to use.
controller_plant = MultibodyPlant(time_step=0.002)
controller_iiwa = AddIiwa(controller_plant)
AddBin(controller_plant, controller_iiwa)
controller_plant.Finalize()

# Add the iiwa controller
iiwa_controller = builder.AddSystem(
    InverseDynamicsController(controller_plant,
                              kp=[5] * num_iiwa_positions,
                              ki=[2] * num_iiwa_positions,
                              kd=[5] * num_iiwa_positions,
                              has_reference_acceleration=False))
iiwa_controller.set_name(model_instance_name + "_controller")
builder.Connect(plant.get_state_output_port(model_instance),
                iiwa_controller.get_input_port_estimated_state())

# Add in the feed-forward torque
adder = builder.AddSystem(Adder(2, num_iiwa_positions))
builder.Connect(iiwa_controller.get_output_port_control(),
                adder.get_input_port(0))
# Use a PassThrough to make the port optional (it will provide zero
# values if not connected).
torque_passthrough = builder.AddSystem(
    PassThrough([0] * num_iiwa_positions))
builder.Connect(torque_passthrough.get_output_port(),
                adder.get_input_port(1))
builder.ExportInput(torque_passthrough.get_input_port(),
                    model_instance_name + "_feedforward_torque")
builder.Connect(adder.get_output_port(),
                plant.get_actuation_input_port(model_instance))

# Add discrete derivative to command velocities.
desired_state_from_position = builder.AddSystem(
    StateInterpolatorWithDiscreteDerivative(
        num_iiwa_positions,
        0.002,
        suppress_initial_transient=True))
desired_state_from_position.set_name(
    model_instance_name + "_desired_state_from_position")
builder.Connect(desired_state_from_position.get_output_port(),
                iiwa_controller.get_input_port_desired_state())
builder.Connect(iiwa_position.get_output_port(),
                desired_state_from_position.get_input_port())

builder.ExportOutput(adder.get_output_port(),
                     model_instance_name + "_torque_commanded")
builder.ExportOutput(adder.get_output_port(),
                     model_instance_name + "_torque_measured")

builder.ExportOutput(
    plant.get_generalized_contact_forces_output_port(
        model_instance), model_instance_name + "_torque_external")

# Add ICP Sphere Finder
sphere_finder = builder.AddSystem(
    ICPSphereFinder())

builder.Connect(builder.GetSystems()[3].point_cloud_output_port(),
                sphere_finder.get_input_port())

# Add integrator to get time
int_source = builder.AddSystem(
    ConstantVectorSource([1.0]))
timer = builder.AddSystem(
    Integrator(1))
builder.Connect(int_source.get_output_port(),
               timer.get_input_port())

# Add sphere trajectory estimator
sphere_traj_est = builder.AddSystem(
    TrajectoryEstimator())
builder.Connect(sphere_finder.get_output_port(),
                sphere_traj_est.get_input_port(0))
builder.Connect(timer.get_output_port(),
                sphere_traj_est.get_input_port(1))

# Add catch planner
catch_planner = builder.AddSystem(
    CatchPlanner(plant))
builder.Connect(sphere_traj_est.get_output_port(),
                catch_planner.get_input_port(0))
builder.Connect(plant.get_body_poses_output_port(),
                catch_planner.get_input_port(1))

# builder.Connect(catch_planner.get_output_port(),
#                 differential_ik.get_input_port(0))

# Add catch controllers and switch
catch_controller = builder.AddSystem(
    CatchIKcontroller(controller_plant))

builder.Connect(catch_planner.get_output_port(0),
               catch_controller.get_input_port(0))
builder.Connect(demux.get_output_port(0),
               catch_controller.get_input_port(1))
builder.Connect(catch_controller.get_output_port(),
               iiwa_position.get_input_port())

# combined catch planner
# catch_ik_planner = builder.AddSystem(
#     CatchIKPlanner(controller_plant))

# builder.Connect(sphere_traj_est.get_output_port(),
#                catch_ik_planner.get_input_port(0))
# builder.Connect(demux.get_output_port(0),
#                catch_ik_planner.get_input_port(1))
# builder.Connect(catch_ik_planner.get_output_port(),
#                iiwa_position.get_input_port())

diagram = builder.Build()

In [22]:
# Automatic sampling of a trajectory

def gen_traj():
    # First sample a starting point visible to the camera
    p_init = np.array([np.random.uniform(-3.5, -2), np.random.uniform(-0.5, 0.5), np.random.uniform(-1, 0)])

    # Then, sample a potential interception point
    theta_int = np.random.uniform(2 * np.pi)
    p_int = np.array([0.5 + 0.5*np.sin(theta_int), 0.75 * np.cos(theta_int), np.random.uniform(-0.25, 0.75)])
    t_int = np.random.uniform(0.5, 1)

    # Calculate initial velocity
    g = np.array([0, 0, 9.8067])
    p_adj = p_int + 0.5 * g * t_int * t_int
    v_init = (p_adj - p_init) / t_int

    return p_init, v_init


# ================================================================================
import time
while True:
    SPHERE_FINDER = prog_find_sphere_iter
    CAM_NOISE = 0.005
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    sphere_context = sphere_finder.GetMyMutableContextFromRoot(context)
    diagram.Publish(context)

    p_init, v_init = gen_traj()
    plant.SetFreeBodyPose(plant_context, proj_body, RigidTransform(RotationMatrix.Identity(), p_init))
    plant.SetFreeBodySpatialVelocity(proj_body, SpatialVelocity([0, 0, 0], v_init), plant_context)

    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)

#     visualizer.StartRecording()
    simulator.AdvanceTo(1.5 if running_as_notebook else 0.1)
    time.sleep(1)
#     visualizer.StopRecording()
#     visualizer.PublishRecording()

Predicted trajectory: [-3.2811323   0.1122749  -0.21819607] [7.49405042 0.25697599 4.39589122]
Intercept at time 0.5008866381367569 [0.47253743 0.24099074 0.75304429]
Predicted trajectory: [-2.74481332 -0.02516499 -0.55557593] [5.55797565 0.43097836 5.07600176]
Intercept at time 0.5778788673002639 [0.46702336 0.2238883  0.73974297]
Predicted trajectory: [-2.0318086   0.05855262 -0.44193839] [3.51341603 0.29555488 3.4404337 ]
Intercept at time 0.5759599164467176 [-0.00822179  0.22878039 -0.08752128]
Predicted trajectory: [-2.81891069  0.40055333 -0.82948718] [ 5.16930357 -0.88184693  4.90602021]
Intercept at time 0.6173724503722966 [ 0.37247493 -0.14387467  0.32981996]
Predicted trajectory: [-2.92483706  0.4315258  -0.97863051] [ 4.29071356 -1.21633114  4.91738847]
Intercept at time 0.6999309744684431 [ 0.07836626 -0.41982204  0.06022598]
Predicted trajectory: [-3.07394398 -0.01918228 -0.90776776] [ 5.63315026 -1.15578681  4.80183895]
Intercept at time 0.5897457375945239 [ 0.24818238 -0

KeyboardInterrupt: 

In [36]:
# Automatic sampling of a trajectory

# First sample a starting point visible to the camera
p_init = np.array([np.random.uniform(-4, -2), np.random.uniform(-0.5, 0.5), np.random.uniform(-1, 0)])

# Then, sample a potential interception point
theta_int = np.random.uniform(2 * np.pi)
p_int = np.array([0.5 + 0.5*np.sin(theta_int), np.cos(theta_int), np.random.uniform(-0.25, 0.75)])
t_int = np.random.uniform(0.5, 1)

# Calculate initial velocity
g = np.array([0, 0, 9.81])
p_adj = p_int + 0.5 * g * t_int * t_int
v_init = (p_adj - p_init) / t_int

print(p_init, v_init, p_int, t_int)


# ================================================================================

#SPHERE_FINDER = ransac_find_sphere_r
CAM_NOISE = 0.003
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
sphere_context = sphere_finder.GetMyMutableContextFromRoot(context)
diagram.Publish(context)

plant.SetFreeBodyPose(plant_context, proj_body, RigidTransform(RotationMatrix.Identity(), p_init))
plant.SetFreeBodySpatialVelocity(proj_body, SpatialVelocity([0, 0, 0], v_init), plant_context)

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)

visualizer.StartRecording()
simulator.AdvanceTo(1.5 if running_as_notebook else 0.1)
visualizer.StopRecording()
visualizer.PublishRecording()

[-2.29102633  0.05219802 -0.32865886] [4.05103646 0.0719881  4.20710184] [ 0.99693105  0.11062598 -0.14519805] 0.8116336182083312
Predicted trajectory: [-2.29117216  0.05021733 -0.32823183] [3.98811712 0.0941999  4.21458086]
Intercept at time 0.6501551981375538 [0.30172291 0.11146188 0.33854758]


In [23]:
# full pipeline results
test_arr = 100 * np.array([1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]
) # no compensation, kp = kd = 50
print(f"${np.mean(test_arr):.2f} \pm {scipy.stats.sem(test_arr):.2f}$")

test_arr = 100 * np.array([0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0]
                         )
print(f"${np.mean(test_arr):.2f} \pm {scipy.stats.sem(test_arr):.2f}$")

$67.95 \pm 5.32$
$56.06 \pm 6.16$
